In [1]:
import importlib
import pandas as pd
import preprocess as ad

importlib.reload(ad)

<module 'preprocess' from '/mnt/data/tad/preprocess.py'>

### Convert log template to log keys
### Just change filename (i.e. ransomware-001) to machine of interest

In [10]:
def linux_file_generator(log_source, filename, df):
    filename = "abnormal" + log_source.lower() + "_" + filename
    
    with open("Dataset/" + log_source + "/" + filename, 'w') as f:
        for event_id in df['Log Key']:
            f.write(str(event_id) + ' ')
        f.write('\n') 

In [3]:
def linux_seq(df_log):

    df_log.Month = pd.to_datetime(df_log.Month, format='%b').dt.month
    df_log['Date'] = df_log['Date'].astype(str)
    df_log['Month'] = df_log['Month'].astype(str)    
    
    searchfor = ['jy|bt|mw']

    norm = df_log[~df_log.Level.str.contains('|'.join(searchfor), regex=True)]
    abnorm = df_log[df_log.Level.str.contains('|'.join(searchfor), regex=True)]
    
    time_seq = '1min'
    # time_seq = '30S'
    # time_seq = '10min'
    
    normal = linux_time(norm, time_seq)
    linux_file_generator("Linux", 'normal_'+time_seq, normal)
    
    abnormal = linux_time(abnorm, time_seq)
    linux_file_generator("Linux", 'abnormal_'+time_seq, abnormal)    
    
    return


In [4]:
def linux_time(df, time_seq):
    df['datetime'] = pd.to_datetime('2020' + "-" + df['Month'] + "-" + df['Date'] + " " + df['Time'])
    df = df[['datetime', 'Log Key']]
    deeplog_df = df.set_index('datetime').resample(time_seq).apply(_custom_resampler).reset_index()
    return deeplog_df

def _custom_resampler(array_like):
    return list(array_like)            

In [41]:
df = pd.read_csv("Spell_result/" + 'linux.log_structured.csv')

# filename = 'ransomware-001'
filename = "mw-virus-003"
searchfor = [filename]

#get logs from machine of interest
df_mw = df[df.Level.str.contains('|'.join(searchfor), regex=True)]

In [42]:
#use this to create sequences according to time
#args: log_source, filename, dataframe containing  logs df_mw
linux_file_generator("Linux", filename, df_mw)

### Split data for federated learning

In [ ]:
ad.federated_split("linux_train", "Linux", 10)